## Build and run a Keras model locally

Assumes images have been preprocessed and converted to tfrecords<br/>
Converts Keras model to Estimator using model_to_estimator<br/>
Saves the model in GCS<br/>
Run prediction method (locally) from the saved model<br/>

In [ ]:
import tensorflow as tf
tf.__version__

## Initialize values

In [ ]:
BUCKET_NAME=''
MODEL_NAME=''
TRAIN_STEPS=1000
BATCH_SIZE = 50
N_EPOCHS = 1

## Create the directories on GCS

In [ ]:
import tensorflow as tf
import os


try:
  ROOT_DIR = 'gs://{}'.format(BUCKET_NAME)
except NameError:
  ROOT_DIR = './tutorial'
  
DATA_DIR = '{}/data'.format(ROOT_DIR)
MODEL_DIR = '{}/{}'.format(ROOT_DIR, MODEL_NAME)
EXPORT_DIR = '{}/{}'.format(ROOT_DIR, MODEL_NAME)
CHECKPOINT_PATH = '{}/checkpoint'.format(MODEL_DIR)

os.environ['BUCKET_NAME']=BUCKET_NAME
os.environ['MODEL_NAME']=MODEL_NAME
  
# Remove CHECKPOINT_DIR if needed
if tf.gfile.IsDirectory(MODEL_DIR):
  tf.logging.info('delete {}'.format(MODEL_DIR))
  tf.gfile.DeleteRecursively(MODEL_DIR)


## Input function using tf.data.Dataset

In [ ]:
def generate_input_fn(file_pattern, mode, batch_size=BATCH_SIZE, count=N_EPOCHS):
  
  def parse_record(serialized_example):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'image': tf.FixedLenFeature([], tf.string),
            'label': tf.FixedLenFeature([], tf.string),            
        })
    # Normalize from [0, 255] to [0.0, 1.0]
    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [28*28]) / 255.0
    label = tf.decode_raw(features['label'], tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10, dtype=tf.int32)
    return image, label

  def input_fn():
    files = tf.data.Dataset.list_files(file_pattern)
    dataset = tf.data.TFRecordDataset(files)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
      dataset = dataset.cache()
      dataset = dataset.shuffle(10000)
      dataset = dataset.repeat(count=count)
      
    dataset = dataset.map(parse_record)
    dataset = dataset.batch(batch_size)
    
    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()

    return features, labels
  
  return input_fn

## Build model and return the estimator and input_signature

In [ ]:
def get_estimator(MODEL_DIR):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(300, activation='relu', input_shape=[28*28]))
  model.add(tf.keras.layers.Dense(100, activation='relu'))  
  model.add(tf.keras.layers.Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.SGD(lr=0.005),
                metrics=['accuracy'])  
  estimator = tf.keras.estimator.model_to_estimator(
    model, model_dir=MODEL_DIR)

  input_signature = model.input.name.split(':')[0]
  
  return estimator, input_signature


## Serving input function

In [ ]:
def get_serving_input_fn(input_signature):
  def preprocess(x):
    return tf.reshape(x, [-1, 28*28]) / 255.0

  def serving_input_fn():
    receiver_tensor = {'X': tf.placeholder(tf.float32, shape=[None, 28, 28])}
    features = {input_signature: tf.map_fn(preprocess, receiver_tensor['X'])}
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensor)
  return serving_input_fn

## Input functions for train, eval, and test datasets

In [ ]:
train_input_fn = generate_input_fn(
    file_pattern='{}/train.tfrecord'.format(DATA_DIR),
    mode=tf.estimator.ModeKeys.TRAIN,
    batch_size=BATCH_SIZE, count=N_EPOCHS)

eval_input_fn = generate_input_fn(
    file_pattern='{}/eval.tfrecord'.format(DATA_DIR),
    mode=tf.estimator.ModeKeys.EVAL, count=1)

test_input_fn = generate_input_fn(
    file_pattern='{}/test.tfrecord'.format(DATA_DIR),
    mode=tf.estimator.ModeKeys.PREDICT)

## Run the training and evaluation using the estimator

In [ ]:
# get the estimator and input signature from the keras model conversion to tf.estimator
estimator, input_signature = get_estimator(MODEL_DIR)

# exporter is invoked during evaluation
exporter = tf.estimator.LatestExporter(
    name='export',
    serving_input_receiver_fn=get_serving_input_fn(input_signature))

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(
    input_fn=eval_input_fn,
    steps=None,
    start_delay_secs=60,
    throttle_secs=60,
    exporters=exporter)

# finally, train and evaluate
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## Show the model location and signature of the serving function

In [ ]:
%%bash

MODEL_LOCATION=$(gsutil ls gs://${BUCKET_NAME}/${MODEL_NAME}/export/ | tail -1)
echo "export model location:" ${MODEL_LOCATION}

saved_model_cli show --dir ${MODEL_LOCATION} --tag_set serve --signature_def serving_default

## Set the values for the export directory and output key

In [ ]:
# get the export model location
export_dir = ""

#get the tensor output key, for example 'dense_3' from the above output
outputs_key=''

## Predict with test data locally from the saved model

In [ ]:
import numpy as np


N_EXAMPLES = 100

train, test = tf.keras.datasets.mnist.load_data()
X_train = train[0][:-5000]
y_train = train[1][:-5000]
X_eval = train[0][-5000:]
y_eval = train[1][-5000:]
X_test = test[0]
y_test = test[1]

predictor_fn = tf.contrib.predictor.from_saved_model(
  export_dir=export_dir, signature_def_key='serving_default')

_X = X_test[:N_EXAMPLES]
_y = y_test[:N_EXAMPLES]

output = predictor_fn({'X': _X})
class_ids = np.argmax(output[outputs_key], axis=2).reshape(-1)

accuracy = np.sum(_y == class_ids)/float(N_EXAMPLES)
print(accuracy)